# ساخت یک مدل رگرسیون: شروع به کار با R و Tidymodels برای مدل‌های رگرسیون


## مقدمه‌ای بر رگرسیون - درس ۱

#### قرار دادن در چشم‌انداز

✅ روش‌های مختلفی برای رگرسیون وجود دارد و انتخاب روش مناسب به نوع پاسخی که به دنبال آن هستید بستگی دارد. اگر بخواهید ارتفاع احتمالی یک فرد با سن مشخص را پیش‌بینی کنید، از `رگرسیون خطی` استفاده می‌کنید، زیرا به دنبال یک **مقدار عددی** هستید. اگر بخواهید بررسی کنید که آیا یک نوع غذا باید به عنوان وگان در نظر گرفته شود یا نه، به دنبال **تخصیص دسته‌بندی** هستید، بنابراین از `رگرسیون لجستیک` استفاده می‌کنید. در ادامه بیشتر درباره رگرسیون لجستیک خواهید آموخت. کمی به سوالاتی که می‌توانید از داده‌ها بپرسید فکر کنید و ببینید کدام یک از این روش‌ها مناسب‌تر است.

در این بخش، شما با [یک مجموعه داده کوچک درباره دیابت](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html) کار خواهید کرد. تصور کنید که می‌خواهید یک درمان برای بیماران دیابتی آزمایش کنید. مدل‌های یادگیری ماشین ممکن است به شما کمک کنند تا تعیین کنید کدام بیماران بهتر به درمان پاسخ می‌دهند، بر اساس ترکیب متغیرها. حتی یک مدل رگرسیون بسیار ساده، وقتی به صورت بصری نمایش داده شود، ممکن است اطلاعاتی درباره متغیرهایی ارائه دهد که به شما در سازماندهی آزمایش‌های بالینی نظری کمک کند.

با این توضیحات، بیایید این کار را شروع کنیم!

<p >
   <img src="../../images/encouRage.jpg"
   width="630"/>
   <figcaption>اثر هنری از @allison_horst</figcaption>

<!--![اثر هنری از \@allison_horst](../../../../../../translated_images/encouRage.e75d5fe0367fb9136b78104baf4e2032a7622bc42a2bc34c0ad36c294eeb83f5.fa.jpg)<br>اثر هنری از @allison_horst-->


## 1. بارگذاری ابزارهای مورد نیاز

برای این کار، به بسته‌های زیر نیاز داریم:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) یک [مجموعه‌ای از بسته‌های R](https://www.tidyverse.org/packages) است که طراحی شده تا علم داده را سریع‌تر، آسان‌تر و جذاب‌تر کند!

-   `tidymodels`: چارچوب [tidymodels](https://www.tidymodels.org/) یک [مجموعه‌ای از بسته‌ها](https://www.tidymodels.org/packages/) برای مدل‌سازی و یادگیری ماشین است.

می‌توانید آنها را با دستور زیر نصب کنید:

`install.packages(c("tidyverse", "tidymodels"))`

اسکریپت زیر بررسی می‌کند که آیا بسته‌های مورد نیاز برای تکمیل این ماژول را دارید یا خیر و در صورتی که برخی از آنها موجود نباشند، آنها را برای شما نصب می‌کند.


In [2]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse, tidymodels)

Loading required package: pacman



حالا، بیایید این بسته‌های فوق‌العاده را بارگذاری کنیم و در جلسه فعلی R خود در دسترس قرار دهیم. (این فقط برای توضیح است، `pacman::p_load()` قبلاً این کار را برای شما انجام داده است)


In [ ]:
# load the core Tidyverse packages
library(tidyverse)

# load the core Tidymodels packages
library(tidymodels)


## 2. مجموعه داده دیابت

در این تمرین، مهارت‌های رگرسیون خود را با پیش‌بینی روی یک مجموعه داده دیابت به نمایش می‌گذاریم. [مجموعه داده دیابت](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt) شامل `442 نمونه` داده مرتبط با دیابت است که دارای 10 متغیر ویژگی پیش‌بینی‌کننده شامل `سن`، `جنسیت`، `شاخص توده بدنی`، `فشار خون متوسط` و `شش اندازه‌گیری سرم خون` و همچنین یک متغیر خروجی `y` است: یک معیار کمی از پیشرفت بیماری یک سال پس از خط پایه.

|تعداد مشاهدات|442|
|----------------------|:---|
|تعداد پیش‌بینی‌کننده‌ها|10 ستون اول عددی و پیش‌بینی‌کننده هستند|
|خروجی/هدف|ستون 11 یک معیار کمی از پیشرفت بیماری یک سال پس از خط پایه است|
|اطلاعات پیش‌بینی‌کننده|- سن به سال
||- جنسیت
||- bmi شاخص توده بدنی
||- bp فشار خون متوسط
||- s1 tc، کلسترول سرم کل
||- s2 ldl، لیپوپروتئین‌های با چگالی کم
||- s3 hdl، لیپوپروتئین‌های با چگالی بالا
||- s4 tch، کلسترول کل / HDL
||- s5 ltg، احتمالاً لگاریتم سطح تری‌گلیسرید سرم
||- s6 glu، سطح قند خون|

> 🎓 به یاد داشته باشید، این یادگیری نظارت‌شده است و ما به یک هدف به نام 'y' نیاز داریم.

قبل از اینکه بتوانید داده‌ها را با R دستکاری کنید، باید داده‌ها را به حافظه R وارد کنید یا یک اتصال به داده‌ها ایجاد کنید که R بتواند از آن برای دسترسی به داده‌ها به صورت از راه دور استفاده کند.

> بسته [readr](https://readr.tidyverse.org/) که بخشی از Tidyverse است، یک روش سریع و کاربرپسند برای خواندن داده‌های مستطیلی به داخل R ارائه می‌دهد.

حالا، بیایید مجموعه داده دیابت را از این آدرس منبع بارگذاری کنیم: <https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html>

همچنین، یک بررسی اولیه روی داده‌های خود با استفاده از `glimpse()` انجام می‌دهیم و 5 ردیف اول را با استفاده از `slice()` نمایش می‌دهیم.

قبل از اینکه جلوتر برویم، بیایید چیزی را معرفی کنیم که اغلب در کد R با آن مواجه خواهید شد 🥁🥁: عملگر پایپ `%>%`

عملگر پایپ (`%>%`) عملیات را به ترتیب منطقی انجام می‌دهد و یک شیء را به جلو به یک تابع یا عبارت فراخوانی منتقل می‌کند. می‌توانید عملگر پایپ را به‌عنوان گفتن "و سپس" در کد خود تصور کنید.


In [ ]:
# Import the data set
diabetes <- read_table2(file = "https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt")


# Get a glimpse and dimensions of the data
glimpse(diabetes)


# Select the first 5 rows of the data
diabetes %>% 
  slice(1:5)

`glimpse()` به ما نشان می‌دهد که این داده شامل ۴۴۲ ردیف و ۱۱ ستون است و تمام ستون‌ها از نوع داده‌ای `double` هستند.

<br>

> `glimpse()` و `slice()` توابعی در [`dplyr`](https://dplyr.tidyverse.org/) هستند. Dplyr که بخشی از Tidyverse است، یک دستور زبان برای دستکاری داده‌ها ارائه می‌دهد که مجموعه‌ای منسجم از افعال را فراهم می‌کند تا به شما در حل رایج‌ترین چالش‌های دستکاری داده کمک کند.

<br>

حالا که داده‌ها را داریم، بیایید روی یک ویژگی (`bmi`) تمرکز کنیم تا این تمرین را انجام دهیم. برای این کار باید ستون‌های مورد نظر را انتخاب کنیم. پس، چطور این کار را انجام دهیم؟

[`dplyr::select()`](https://dplyr.tidyverse.org/reference/select.html) به ما امکان می‌دهد ستون‌های یک فریم داده را *انتخاب* کنیم (و در صورت نیاز نام آن‌ها را تغییر دهیم).


In [ ]:
# Select predictor feature `bmi` and outcome `y`
diabetes_select <- diabetes %>% 
  select(c(bmi, y))

# Print the first 5 rows
diabetes_select %>% 
  slice(1:10)

## ۳. داده‌های آموزش و آزمایش

در یادگیری نظارت‌شده، معمولاً داده‌ها را به دو زیرمجموعه *تقسیم* می‌کنند؛ یک مجموعه (که معمولاً بزرگ‌تر است) برای آموزش مدل و یک مجموعه کوچک‌تر برای "نگه‌داشتن" و بررسی عملکرد مدل.

حالا که داده‌ها آماده هستند، می‌توانیم ببینیم آیا یک ماشین می‌تواند به ما کمک کند تا یک تقسیم منطقی بین اعداد این مجموعه داده پیدا کنیم. می‌توانیم از بسته [rsample](https://tidymodels.github.io/rsample/) که بخشی از چارچوب Tidymodels است استفاده کنیم تا یک شیء ایجاد کنیم که اطلاعات مربوط به *چگونگی* تقسیم داده‌ها را در خود داشته باشد. سپس با استفاده از دو تابع دیگر از rsample، می‌توانیم مجموعه‌های آموزش و آزمایش ایجادشده را استخراج کنیم:


In [ ]:
set.seed(2056)
# Split 67% of the data for training and the rest for tesing
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.67)

# Extract the resulting train and test sets
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)

# Print the first 3 rows of the training set
diabetes_train %>% 
  slice(1:10)

## 4. آموزش یک مدل رگرسیون خطی با Tidymodels

حالا آماده‌ایم مدل خود را آموزش دهیم!

در Tidymodels، شما مدل‌ها را با استفاده از `parsnip()` و مشخص کردن سه مفهوم تعریف می‌کنید:

-   **نوع مدل** مدل‌ها را از یکدیگر متمایز می‌کند، مانند رگرسیون خطی، رگرسیون لجستیک، مدل‌های درخت تصمیم و غیره.

-   **حالت مدل** شامل گزینه‌های رایج مانند رگرسیون و طبقه‌بندی است؛ برخی از انواع مدل‌ها از هر دو حالت پشتیبانی می‌کنند، در حالی که برخی فقط یک حالت دارند.

-   **موتور مدل** ابزار محاسباتی است که برای برازش مدل استفاده خواهد شد. اغلب این‌ها بسته‌های R هستند، مانند **`"lm"`** یا **`"ranger"`**

این اطلاعات مدل در یک مشخصات مدل ثبت می‌شود، پس بیایید یکی بسازیم!


In [ ]:
# Build a linear model specification
lm_spec <- 
  # Type
  linear_reg() %>% 
  # Engine
  set_engine("lm") %>% 
  # Mode
  set_mode("regression")


# Print the model specification
lm_spec

پس از اینکه یک مدل *مشخص* شد، می‌توان مدل را با استفاده از تابع [`fit()`](https://parsnip.tidymodels.org/reference/fit.html) تخمین زده یا آموزش داد. این کار معمولاً با استفاده از یک فرمول و مقداری داده انجام می‌شود.

`y ~ .` به این معناست که ما `y` را به عنوان مقدار پیش‌بینی‌شده/هدف براساس تمام پیش‌بینی‌کننده‌ها/ویژگی‌ها یعنی `.` برازش می‌کنیم (در این مثال، ما فقط یک پیش‌بینی‌کننده داریم: `bmi`).


In [ ]:
# Build a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>%
  set_mode("regression")


# Train a linear regression model
lm_mod <- lm_spec %>% 
  fit(y ~ ., data = diabetes_train)

# Print the model
lm_mod

از خروجی مدل می‌توان ضرایب یاد گرفته شده در طول آموزش را مشاهده کرد. این ضرایب نشان‌دهنده ضرایب خط برازش بهینه هستند که کمترین خطای کلی بین متغیر واقعی و پیش‌بینی‌شده را به ما می‌دهند.
<br>

## ۵. پیش‌بینی روی مجموعه تست

حالا که یک مدل آموزش داده‌ایم، می‌توانیم از آن برای پیش‌بینی پیشرفت بیماری y در مجموعه داده تست استفاده کنیم با استفاده از [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). این پیش‌بینی برای رسم خط بین گروه‌های داده استفاده خواهد شد.


In [ ]:
# Make predictions for the test set
predictions <- lm_mod %>% 
  predict(new_data = diabetes_test)

# Print out some of the predictions
predictions %>% 
  slice(1:5)

هورا! 💃🕺 ما به‌تازگی یک مدل آموزش دادیم و از آن برای پیش‌بینی استفاده کردیم!

هنگام انجام پیش‌بینی‌ها، روش معمول در tidymodels این است که همیشه یک tibble/داده‌فریم از نتایج با نام‌های استاندارد برای ستون‌ها تولید شود. این کار ترکیب داده‌های اصلی و پیش‌بینی‌ها را در قالبی قابل استفاده برای عملیات بعدی مانند رسم نمودار آسان می‌کند.

`dplyr::bind_cols()` به‌طور کارآمد چندین داده‌فریم را به‌صورت ستونی ترکیب می‌کند.


In [ ]:
# Combine the predictions and the original test set
results <- diabetes_test %>% 
  bind_cols(predictions)


results %>% 
  slice(1:5)

## 6. نمایش نتایج مدل‌سازی

حالا وقتشه که این رو به صورت تصویری ببینیم 📈. ابتدا یک نمودار پراکندگی از تمام مقادیر `y` و `bmi` در مجموعه تست ایجاد می‌کنیم، سپس با استفاده از پیش‌بینی‌ها یک خط در مناسب‌ترین مکان، بین گروه‌بندی‌های داده‌های مدل رسم می‌کنیم.

زبان R چندین سیستم برای ساخت نمودار دارد، اما `ggplot2` یکی از زیباترین و انعطاف‌پذیرترین‌هاست. این سیستم به شما امکان می‌دهد نمودارها را با **ترکیب اجزای مستقل** بسازید.


In [ ]:
# Set a theme for the plot
theme_set(theme_light())
# Create a scatter plot
results %>% 
  ggplot(aes(x = bmi)) +
  # Add a scatter plot
  geom_point(aes(y = y), size = 1.6) +
  # Add a line plot
  geom_line(aes(y = .pred), color = "blue", size = 1.5)

✅ کمی به این موضوع فکر کنید که اینجا چه اتفاقی در حال رخ دادن است. یک خط مستقیم از میان تعداد زیادی نقطه کوچک داده عبور می‌کند، اما دقیقاً چه کاری انجام می‌دهد؟ آیا می‌توانید ببینید که چگونه باید بتوانید از این خط برای پیش‌بینی محل قرارگیری یک نقطه داده جدید و نادیده گرفته‌شده نسبت به محور y نمودار استفاده کنید؟ سعی کنید کاربرد عملی این مدل را با کلمات توضیح دهید.

تبریک می‌گویم، شما اولین مدل رگرسیون خطی خود را ساختید، با آن یک پیش‌بینی انجام دادید و آن را در یک نمودار نمایش دادید!



---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما برای دقت تلاش می‌کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادقتی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، ترجمه حرفه‌ای انسانی توصیه می‌شود. ما هیچ مسئولیتی در قبال سوءتفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
